In [4]:
!git clone https://github.com/nqh-tq-32052503/Moal.git

Cloning into 'Moal'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (124/124), done.
remote: Total 147 (delta 55), reused 51 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (147/147), 455.07 KiB | 11.97 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [6]:
import os
os.listdir("/content/")

['.config', 'Moal', 'sample_data']

In [7]:
%cd /content/Moal

/content/Moal
